# Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
from gazprom.mt.pricing.models.swing.fixed_price import FixedPriceSwing
from gazprom.mt.pricing.models.swing.test.swing_market_data import build_swing
from gazprom.mt.pricing.models.exercise import FixedDailyOffset

C:\dev\bin\Anaconda2\envs\1.8.6.post502\lib\site-packages\gazprom\mt\pricing\models\lsmc\_exercise.py:4: UserWarning: gazprom.mt.pricing.models.lsmc._exercise is deprecated. Please use gazprom.mt.pricing.models.exercise instead.
  warn("gazprom.mt.pricing.models.lsmc._exercise is deprecated. Please use "


In [3]:
from gazprom.mt.pricing.environment.environment import Environment
from gazprom.mt.pricing.environment.market_data import CurveData
from gazprom.mt.pricing.models.timeseries.gbmsimulator.volatility_model import VolModelBlended
from gazprom.mt.pricing.models.timeseries.market_data import MonthlyBlendedData

# Payoff

In [4]:
mkt = 'TTF'
ccy = 'EUR'

start_date = pd.datetime(2017, 3, 1)
end_date = pd.datetime(2017, 3, 20)
exercise_dates = pd.date_range(start_date, end_date, freq='D')
tcq_max = 1.0
tcq_min = 0.0
dcq_max = 1.0
dcq_min = 0.0

strike = 25.5

kwargs_payoff = {'strike': strike, 
                 'dcq_max': dcq_max, 'dcq_min': dcq_min, 'early_exercise': FixedDailyOffset(), 
                 'market': mkt, 'ccy': ccy, 'payoff_class': FixedPriceSwing}

In [5]:
payoff = build_swing('call', 'buy', start_date, end_date, tcq_max, tcq_min, **kwargs_payoff)

In [6]:
print(payoff)

FixedPriceSwing [buy, EUR]: market=TTF, type=call,start_date=01-Mar-2017, end_date=20-Mar-2017
# constraints=SwingConstraints(_dcq_min=0.0, _tcq_min=0.0, _tcq_max=1.0, _dcq_max=1.0)
# strike=
2017-03-01    25.5
2017-03-02    25.5
2017-03-03    25.5
2017-03-04    25.5
2017-03-05    25.5
2017-03-06    25.5
2017-03-07    25.5
2017-03-08    25.5
2017-03-09    25.5
2017-03-10    25.5
2017-03-11    25.5
2017-03-12    25.5
2017-03-13    25.5
2017-03-14    25.5
2017-03-15    25.5
2017-03-16    25.5
2017-03-17    25.5
2017-03-18    25.5
2017-03-19    25.5
2017-03-20    25.5
Freq: D, dtype: float64


# Environment

In [24]:
aux = 23.0 + 10.0 * np.sin(2. * np.pi * np.arange(1, 21) / 20);
# pd.Series(aux[0:1] + aux, 
#                index=pd.date_range(valuation_date, pd.datetime(2017, 3, 20)))
np.concatenate((aux[0:1], aux))

array([ 26.09016994,  26.09016994,  28.87785252,  31.09016994,
        32.51056516,  33.        ,  32.51056516,  31.09016994,
        28.87785252,  26.09016994,  23.        ,  19.90983006,
        17.12214748,  14.90983006,  13.48943484,  13.        ,
        13.48943484,  14.90983006,  17.12214748,  19.90983006,  23.        ])

In [25]:
valuation_date = pd.datetime(2017, 2, 28)

aux = 23.0 + 10.0 * np.sin(2. * np.pi * np.arange(1, 21) / 20);
aux = np.concatenate((aux[0:1], aux));
# fc = pd.Series(25.5, index=pd.date_range(valuation_date, pd.datetime(2017, 3, 20)))
fc = pd.Series(aux, index=pd.date_range(valuation_date, pd.datetime(2017, 3, 20)))

fc.loc[pd.datetime(2017, 3, 7)] = 33.6
fc.loc[pd.datetime(2017, 3, 14)] = 33.6
fc.loc[pd.datetime(2017, 3, 19)] = 33.6

ir = pd.Series(0.0245, index=exercise_dates)

ctc = pd.date_range(pd.datetime(2017, 1, 1), pd.datetime(2017, 4, 1), freq='MS')
ctc_expiries = pd.Series(ctc - pd.datetools.Day(), index=ctc)
option_expiries = pd.Series(ctc - pd.datetools.MonthEnd(), index=ctc)

volm = pd.Series(0.21, index=ctc)
volb = pd.Series(0.45, index=ctc)

env = Environment(valuation_date, ccy)
env.add_price(ccy, CurveData(ir))
env.add_price(mkt, CurveData(fc))
# env.add_expiry_option(mkt, option_expiries)

mbdata = MonthlyBlendedData(vol_monthly=volm, vol_blended=volb, option_expiry=option_expiries)
env.add_vol(mkt, mbdata, VolModelBlended)

In [26]:
print(env)

Environment (intraday=False):
## _STATE
- currency: IdentityData: data_args={'data_type': <type 'basestring'>}, data:
EUR
- intraday: IdentityData: data_args={'data_type': <type 'bool'>}, data:
False
- pricing_date: IdentityData: data_args={'data_type': <type 'datetime.date'>}, data:
2017-02-28 00:00:00
## TTF
- vol_params: IdentityData: data_args={'data_type': <type 'object'>}, data:
{'offset': 14}
- vol_model: VolModelFunction: vol_kwargs={'builder': 'default', 'calibrator': None}, vol_class=<class 'gazprom.mt.pricing.models.timeseries.gbmsimulator.volatility_model.VolModelBlended'>, market_data=<gazprom.mt.pricing.models.timeseries.market_data.MonthlyBlendedData object at 0x000000000CFE38D0>, builder=<gazprom.mt.pricing.environment.data_builders.DefaultDataBuilder object at 0x000000000CFE3550>, data_args={}, data:
(KeySpec(key_type=pricing_date, apply_early=False, key=_STATE), KeySpec(key_type=vol_monthly, apply_early=False, key=TTF), KeySpec(key_type=vol_blended, apply_early=False,

# Valuation

In [35]:
simulator_params = {'seed': 12321312, 'simulator': 'SimulatorOneFactor'}
lsmc_params=dict(nsims_fwd=30000, nsims_bwd=30000)

res = payoff.pv_greeks(env, simulator_params=simulator_params, lsmc_params=lsmc_params, debug=True, intrinsic=True)

In [36]:
res.keys()

['sims',
 'cash_strike',
 'pv',
 'actions_paths',
 'triggers',
 'lsmc_mesh',
 'fwd_sims',
 'cash',
 'actions',
 'spot_sims',
 'delta',
 'theta',
 'price_model',
 'cashflows']

In [37]:
res['pv']

SV(      8.0962,     0.0000)

In [34]:
res['delta']

SVFrame.value:
                  TTF
2017-03-01  0.000000
2017-03-02  0.000000
2017-03-03  0.000000
2017-03-04  0.000000
2017-03-05  0.000000
2017-03-06  0.000000
2017-03-07  0.479796
2017-03-08  0.000000
2017-03-09  0.000000
2017-03-10  0.000000
2017-03-11  0.000000
2017-03-12  0.000000
2017-03-13  0.000000
2017-03-14  0.134808
2017-03-15  0.000000
2017-03-16  0.000000
2017-03-17  0.000000
2017-03-18  0.000000
2017-03-19  0.384843
2017-03-20  0.000000
SVFrame.err:
                  TTF
2017-03-01  0.000000
2017-03-02  0.000000
2017-03-03  0.000000
2017-03-04  0.000000
2017-03-05  0.000000
2017-03-06  0.000000
2017-03-07  0.002827
2017-03-08  0.000000
2017-03-09  0.000000
2017-03-10  0.000000
2017-03-11  0.000000
2017-03-12  0.000000
2017-03-13  0.000000
2017-03-14  0.001870
2017-03-15  0.000000
2017-03-16  0.000000
2017-03-17  0.000000
2017-03-18  0.000000
2017-03-19  0.002971
2017-03-20  0.000000

In [133]:
res['vega_blended']

SVFrame.value:
                  TTF
2017-03-01  0.000000
2017-03-02  0.000000
2017-03-03  0.000000
2017-03-04  0.000000
2017-03-05  0.000000
2017-03-06  0.000000
2017-03-07 -0.008297
2017-03-08  0.000000
2017-03-09  0.000000
2017-03-10  0.000000
2017-03-11  0.000000
2017-03-12  0.000000
2017-03-13  0.000000
2017-03-14 -0.005429
2017-03-15  0.000000
2017-03-16  0.000000
2017-03-17  0.000000
2017-03-18  0.000000
2017-03-19  0.013941
2017-03-20  0.000000
SVFrame.err:
                  TTF
2017-03-01  0.000000
2017-03-02  0.000000
2017-03-03  0.000000
2017-03-04  0.000000
2017-03-05  0.000000
2017-03-06  0.000000
2017-03-07  0.000090
2017-03-08  0.000000
2017-03-09  0.000000
2017-03-10  0.000000
2017-03-11  0.000000
2017-03-12  0.000000
2017-03-13  0.000000
2017-03-14  0.000245
2017-03-15  0.000000
2017-03-16  0.000000
2017-03-17  0.000000
2017-03-18  0.000000
2017-03-19  0.000236
2017-03-20  0.000000

In [134]:
res['actions']

2017-03-01    SV(     -0.0000,     0.0000)
2017-03-02    SV(     -0.0000,     0.0000)
2017-03-03    SV(     -0.0000,     0.0000)
2017-03-04    SV(     -0.0000,     0.0000)
2017-03-05    SV(     -0.0000,     0.0000)
2017-03-06    SV(     -0.0000,     0.0000)
2017-03-07    SV(      0.5201,     0.0029)
2017-03-08    SV(     -0.0000,     0.0000)
2017-03-09    SV(     -0.0000,     0.0000)
2017-03-10    SV(     -0.0000,     0.0000)
2017-03-11    SV(     -0.0000,     0.0000)
2017-03-12    SV(     -0.0000,     0.0000)
2017-03-13    SV(     -0.0000,     0.0000)
2017-03-14    SV(      0.1586,     0.0021)
2017-03-15    SV(     -0.0000,     0.0000)
2017-03-16    SV(     -0.0000,     0.0000)
2017-03-17    SV(     -0.0000,     0.0000)
2017-03-18    SV(     -0.0000,     0.0000)
2017-03-19    SV(      0.3213,     0.0027)
2017-03-20    SV(     -0.0000,     0.0000)
Freq: D, dtype: object

In [135]:
res['price_model'].vol_model

VolModelBlended(_vol_blended=2017-01-01    0.45
2017-02-01    0.45
2017-03-01    0.45
2017-04-01    0.45
Freq: MS, dtype: float64, _valid_index=DatetimeIndex(['2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01'], dtype='datetime64[ns]', freq='MS'), _blended_offset=14, _monthly_expiries=2017-01-01   2016-12-31
2017-02-01   2017-01-31
2017-03-01   2017-02-28
2017-04-01   2017-03-31
Freq: MS, dtype: datetime64[ns], _vol_monthly=2017-01-01    0.21
2017-02-01    0.21
2017-03-01    0.21
2017-04-01    0.21
Freq: MS, dtype: float64, implied_vols_db=2017-01-01    0.45
2017-02-01    0.45
2017-03-01    0.45
2017-04-01    0.45
Freq: MS, dtype: float64)

In [136]:
res['price_model'].vol_model.terminal_vol(valuation_date, payoff.contract_dates, payoff.contract_dates)

(2017-03-01    0.45
 2017-03-02    0.45
 2017-03-03    0.45
 2017-03-04    0.45
 2017-03-05    0.45
 2017-03-06    0.45
 2017-03-07    0.45
 2017-03-08    0.45
 2017-03-09    0.45
 2017-03-10    0.45
 2017-03-11    0.45
 2017-03-12    0.45
 2017-03-13    0.45
 2017-03-14    0.45
 2017-03-15    0.45
 2017-03-16    0.45
 2017-03-17    0.45
 2017-03-18    0.45
 2017-03-19    0.45
 2017-03-20    0.45
 Freq: D, dtype: float64, {'theta': 2017-03-01    0.000000e+00
  2017-03-02    0.000000e+00
  2017-03-03    0.000000e+00
  2017-03-04    5.631946e-15
  2017-03-05    0.000000e+00
  2017-03-06    3.754630e-15
  2017-03-07    3.218255e-15
  2017-03-08    2.815973e-15
  2017-03-09    2.503087e-15
  2017-03-10    0.000000e+00
  2017-03-11    0.000000e+00
  2017-03-12    0.000000e+00
  2017-03-13    0.000000e+00
  2017-03-14    0.000000e+00
  2017-03-15    0.000000e+00
  2017-03-16    0.000000e+00
  2017-03-17    0.000000e+00
  2017-03-18    1.251543e-15
  2017-03-19    0.000000e+00
  2017-03-20   

In [137]:
res['spot_sims'].mean(axis=1)

simulation_dates
2017-03-01    25.500000
2017-03-02    25.499965
2017-03-03    25.499750
2017-03-04    25.499568
2017-03-05    25.499680
2017-03-06    25.499478
2017-03-07    33.599326
2017-03-08    25.499749
2017-03-09    25.499560
2017-03-10    25.499199
2017-03-11    25.499200
2017-03-12    25.499713
2017-03-13    25.500249
2017-03-14    33.600473
2017-03-15    25.500790
2017-03-16    25.500629
2017-03-17    25.500701
2017-03-18    25.501452
2017-03-19    33.602415
2017-03-20    25.501696
Freq: D, dtype: float64

In [138]:
res['delta'].value.sum()

TTF    0.999465
dtype: float64

In [139]:
set(res['actions_paths'].sum(axis=0))

{1.0}

In [140]:
res['actions_paths'].sum(axis=1)

2017-03-01        0
2017-03-02        0
2017-03-03        0
2017-03-04        0
2017-03-05        0
2017-03-06        0
2017-03-07    15603
2017-03-08        0
2017-03-09        0
2017-03-10        0
2017-03-11        0
2017-03-12        0
2017-03-13        0
2017-03-14     4757
2017-03-15        0
2017-03-16        0
2017-03-17        0
2017-03-18        0
2017-03-19     9640
2017-03-20        0
Freq: D, dtype: float64

In [142]:
[15603/30000., 4757/30000., 9640/30000.]

[0.5201, 0.15856666666666666, 0.32133333333333336]